In [ ]:
!pip install --quiet openai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.1 MB/s eta 0:00:00


# 1. Vision Analysis

In [ ]:
from openai import OpenAI
import base64
import os
from google.colab import userdata

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as f:
    return base64.b64encode(f.read()).decode('utf-8')

In [ ]:
vision_prompt = """
  You are an extremely knowledgeable Image Analyst.
  Your expertise includes analyzing and understanding every detail of the provided image.

  Task:
  Your task is to examine the following image in detail and provide a detailed, factual, and accurate explanation of what the image depicts.
"""

In [ ]:
image_path = '/content/img2.jpg'
base64_image = encode_image(image_path)

In [ ]:
client = OpenAI()
model="gpt-4-vision-preview"
max_tokens=1200

In [ ]:
messages = [
    {
        "role" : "user",
        "content":[
            {'type' : 'text',
             "text" : f'{vision_prompt}'},
            {
                "type" : "image_url",
                "image_url" : f"data:image/jpeg;base64,{base64_image}"
            }
        ]
    }
]

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=max_tokens
)

In [ ]:
vision_response = response.choices[0].message.content

In [ ]:
vision_response

"The image provided shows a picturesque landscape scene along a coastline, likely captured during the golden hour, as evidenced by the warm light and long shadows adding depth to the landscape. The prominent feature of this photograph is an arched bridge spanning across a steep ravine or chasm. The bridge appears to be of a historic or significant architectural design, possibly situated along a scenic route, reminiscent of the Bixby Creek Bridge on the Big Sur coast of California, although without explicit identifying features, such an identification remains speculative.\n\nThe sweeping coastline consists of rugged cliffs descending into a foamy and active ocean. Waves crash against the shore, where a sandy beach is nestled at the base of the cliffs. The landscape beyond the bridge rolls into hills and mountains, covered with greenery that suggests a temperate climate conducive to lush vegetation.\n\nThe sky is clear with only a few wispy clouds, indicating fair weather. This creates a

# Conversational Agent

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [ ]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    print(now)
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [ ]:
template = '''
You are a skilled conversational assistant
You are sarcastic, and witty, with a sense of creativity.

You know the date and time based on the information provided below.
date: {date}

You also keep track of previous conversations using the chat history.
Chat History: {history}

Your expertise includes understanding each detail of the context provided below.
Context: {response}

Your Task:
Understand the user question given below.

User Question: {input}

Then, respond appropriately to the user's question while giving suggestions based on the context.
While responding, keep your response short and clear, and also use Yes, No, Sure, wherever necessary.
'''

In [ ]:
prompt = PromptTemplate(
    input_variables=["history","input"],
    template=template,
    partial_variables={ "response": vision_response, "date" : _get_datetime}
)

In [ ]:
# prompt.format(input = "Can i park my car here?", history = memory)

In [ ]:
llm = ChatOpenAI(model = 'gpt-4', temperature=1.0)

In [ ]:
chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

In [ ]:
# query = "Can i park my car here now?"
# response = chain.run(query)
# response

In [ ]:
# query = "When can i park my car here today?"
# response = chain.invoke(query)
# response['response']

In [ ]:
# query = "What did i previously ask?"
# response = chain.invoke(query)
# response['response']

In [ ]:
# memory.clear()

In [ ]:
# query = "Where was this picture taken?"
# response = chain.run(query)
# response

In [ ]:
query = "What is the speed limit of the road seen in the images?"
response = chain.invoke(query)
response['response']

2024-03-01 16:15:24.696454


"Oh, the wonders of your attention to detail! Unfortunately, the image description doesn't mention any specifics about the speed limit on the gorgeous road captured. It is indeed a world hidden from sight. Thus, darlin', we can't determine the speed limit. However, should you ever find yourself on a scenic route similar to this, like Big Sur in California, the speed limit varies typically between 25 to 55 mph, depending on local conditions and regulations. Remember, safety always comes before the urgency to imprint these mesmerizing landscapes in your memory!"

In [ ]:
memory.clear()